In [6]:
# Import the QICK drivers and auxiliary libraries
from qick import *
%pylab inline

import xrfclk
from qick.ipq_pynq_utils import ipq_pynq_utils
# import spidev # pip package: "pip install spidev" or "apt install python3-spidev"
from ipq_pynq_utils import spidev # our reimplementation of spidev
from importlib.resources import files

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:
# how to use xrfclk to set clocks

# xrfclk.set_ref_clks(lmk_freq=245.76, lmx_freq=245.76*2)

In [8]:
# how to use IPQ utils to set clocks

board = ipq_pynq_utils.ZCU208Board()

regs_lmk = ipq_pynq_utils.clock_models.LMK04828B(10, 10, 10, 160)
with files("xrfclk").joinpath("LMK04828_245.76.txt").open() as f:
    regs_lmk.init_from_file(f)
regs_lmx = ipq_pynq_utils.clock_models.LMX2594(245.76)
with files("xrfclk").joinpath("LMX2594_491.52.txt").open() as f:
    regs_lmx.init_from_file(f)
    
ipq_pynq_utils.ZCU208Board._write_registers(board.spi_lmk, [0x90] + regs_lmk.get_register_dump())
ipq_pynq_utils.ZCU208Board._write_registers(board.spi_adc, 
                                            [0x700000, 0x6f0000, 0x6e0000] + regs_lmx.get_register_dump())
ipq_pynq_utils.ZCU208Board._write_registers(board.spi_dac, 
                                            [0x700000, 0x6f0000, 0x6e0000] + regs_lmx.get_register_dump())

In [9]:
# force init clocks to make sure we reset the clock chips
soc = QickSoc("/data/fw/2024-12-05_216_tprocv2r21_demo/qick_216.bit", force_init_clks=True)
soccfg = soc
print(soccfg)

resetting clocks: 245.76 491.52
QICK running on ZCU216, software version 0.2.301

Firmware configuration (built Thu Dec  5 14:57:40 2024):

	Global clocks (MHz): tProcessor 614.400, RF reference 245.760

	6 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 16384 samples (1.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_sg_mux8_v1 - envelope memory 0 samples (0.000 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_sg_int4_v2 - envelope memory 16384 samples (38.095 us)
		fs=6881.280 MHz, fabric=430.080 MHz, 32-bit DDS, range=1720.320 MHz
		DAC tile 3, blk 0 is 0_231, on JHC3
	3:	axis_sg_int4_v2 - envelope memory 16384 samples (38.095 us)
		fs=6881.280 MHz, fabric=430.080 MHz, 32-bit DDS, range=1720.320 MHz
		DAC tile 3, blk 1 is 1_231, on JHC4
	4:	axis_signal_gen_v6 - envelope memory 8192 samples (0.833 us)
		fs=9830.400 MH

In [10]:
def write_reg(spi, regdev, regvals):
    for k,v in regvals.items():
        getattr(regdev, k).value = v
        
    for addr in regdev.find_addrs(regvals.keys()):
        msg = regdev.registers_by_addr[addr].get_raw().to_bytes(length=3, byteorder='big')
        spi.writebytes(msg)

def read_reg(spi, dummydev, regnames):
    for addr in dummydev.find_addrs(regnames):
        msg = (0x800000 + (addr<<dummydev.dw)).to_bytes(length=3, byteorder='big')
#         print(msg)
        a = spi.xfer(msg)
#         print(a)
        res = int.from_bytes(a, byteorder='big')

        dummydev.registers_by_addr[addr].parse(res)

    readbacks = {}
    for regname in regnames:
        readbacks[regname] = getattr(dummydev, regname).value
    return readbacks

In [13]:
dummy_lmk = ipq_pynq_utils.clock_models.LMK04828B(10, 10, 10, 160)
regs_lmk = ipq_pynq_utils.clock_models.LMK04828B(10, 10, 10, 160)
with files("xrfclk").joinpath("LMK04828_245.76.txt").open() as f:
    regs_lmk.init_from_file(f)

soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x2, 0xff)
with spidev.SpiDev(1, 1) as spi:
    
    spi.bits_per_word = 8
    spi.max_speed_hz = 100000

    # set SPI output type to push-pull; default is 6 (open-drain) which is incorrect
    write_reg(spi, regs_lmk, {"PLL1_LD_TYPE": 3})

    readbacks = read_reg(spi, dummy_lmk, ['RB_DAC_VALUE', 
                                    'RB_CLKin0_LOS', 
                                    'RB_CLKin1_LOS', 
                                    'RB_PLL1_LD', 
                                    'RB_PLL2_LD', 
                                    'RB_PLL1_LD_LOST', 
                                    'RB_PLL2_LD_LOST'])
    
    for k,v in readbacks.items():
        print(k,v)
    
    # clear the lock-lost flags
    write_reg(spi, regs_lmk, {"CLR_PLL1_LD_LOST": 1,
                             "CLR_PLL2_LD_LOST": 1})
    write_reg(spi, regs_lmk, {"CLR_PLL1_LD_LOST": 0,
                             "CLR_PLL2_LD_LOST": 0})

RB_DAC_VALUE 512
RB_CLKin0_LOS 0
RB_CLKin1_LOS 0
RB_PLL1_LD 1
RB_PLL2_LD 1
RB_PLL1_LD_LOST 0
RB_PLL2_LD_LOST 0


In [14]:
dummy_lmx = ipq_pynq_utils.clock_models.LMX2594(245.76)
regs_lmx = ipq_pynq_utils.clock_models.LMX2594(245.76)
with files("xrfclk").joinpath("LMX2594_491.52.txt").open() as f:
    regs_lmx.init_from_file(f)

soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x0, 0xff)

with spidev.SpiDev(1, 3) as spi:
    spi.bits_per_word = 8
    spi.max_speed_hz = 100000

    # set output to SPI readback; default is 1 (lock detect) which drives the LED
    write_reg(spi, regs_lmx, {"MUXOUT_LD_SEL": 0})
    readbacks = read_reg(spi, dummy_lmx, ['rb_LD_VTUNE',
                                    'rb_VCO_SEL',
                                    'rb_VCO_CAPCTRL',
                                    'rb_VCO_DACISET'])
    
    for k,v in readbacks.items():
        print(k,v)
        
    # switch back to indicating lock-detect with the LED
    write_reg(spi, regs_lmx, {"MUXOUT_LD_SEL": 1})

rb_LD_VTUNE 2
rb_VCO_SEL 1
rb_VCO_CAPCTRL 99
rb_VCO_DACISET 308


In [15]:
dummy_lmx = ipq_pynq_utils.clock_models.LMX2594(245.76)
regs_lmx = ipq_pynq_utils.clock_models.LMX2594(245.76)
with files("xrfclk").joinpath("LMX2594_491.52.txt").open() as f:
    regs_lmx.init_from_file(f)

soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x1, 0xff)

with spidev.SpiDev(1, 2) as spi:
    spi.bits_per_word = 8
    spi.max_speed_hz = 100000

    # set output to SPI readback; default is 1 (lock detect) which drives the LED
    write_reg(spi, regs_lmx, {"MUXOUT_LD_SEL": 0})
    readbacks = read_reg(spi, dummy_lmx, ['rb_LD_VTUNE',
                                    'rb_VCO_SEL',
                                    'rb_VCO_CAPCTRL',
                                    'rb_VCO_DACISET'])
    
    for k,v in readbacks.items():
        print(k,v)
        
    # switch back to indicating lock-detect with the LED
    write_reg(spi, regs_lmx, {"MUXOUT_LD_SEL": 1})
    

rb_LD_VTUNE 2
rb_VCO_SEL 1
rb_VCO_CAPCTRL 101
rb_VCO_DACISET 307


In [ ]:
# print(ipq_lmk.PLL1_LD_MUX)
# print(ipq_lmk.PLL1_LD_TYPE)
# print(ipq_adc.MUXOUT_LD_SEL)

In [ ]:
soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x2, 0xff)
s = board.spi_lmk

addr = 388
msg = [((addr>>8) & 0xFF) + (1<<7), addr & 0xFF, 0]
print(msg)
a = s.xfer(msg)
print(hex(a[2]))

In [ ]:
soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x0, 0xff)

s = board.spi_adc
addr = 110
msg = [addr + (1<<7), 0, 0]
print(msg)
a = s.xfer(msg)
print(hex((a[1] << 8)+a[2]))

In [ ]:
soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x1, 0xff)

s = board.spi_dac
addr = 110
msg = [addr + (1<<7), 0, 0]
print(msg)
a = s.xfer(msg)
print(a)

In [ ]:
for k,v in dummy_lmk.registers_by_addr.items():
    for x in v.fields:
        if x.name != 'CONST':
            print(k, x.name)

In [ ]:
dir(dummy_lmk)

In [ ]:
# for x in dummy_reg.fields:
#     print(x.name, x.value)

In [ ]:
# dummy_lmk.registers_by_addr[0x182].fields[0].name

In [ ]:
soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x2, 0xff)
s = board.spi_lmk

for addr in [0x182, 0x183, 0x184, 0x185]:
    dummy_reg = dummy_lmk.registers_by_addr[addr]
    # print(dummy_reg)
    # for x in dummy_reg.fields:
    #     print(x.name, x.value)
    
    msg = [((addr>>8) & 0xFF) + (1<<7), addr & 0xFF, 0]
#     print(msg)
    a = s.xfer(msg)
    res = a[2]
    print("0x%x: 0x%x"%(addr,res))
    
    dummy_reg.parse(res)
    for f in dummy_reg.fields:
        if f.name != 'CONST':
            print("%s: 0x%x"%(f.name, f.value))


In [ ]:
xrfclk.lmk_devices=[]
xrfclk.lmx_devices=[]
xrfclk.xrfclk._find_devices()


In [ ]:
xrfclk.lmx_devices

In [ ]:
xrfclk.xrfclk._read_tics_output()

In [ ]:
xrf_conf = xrfclk.xrfclk._Config['lmk04828'][245.76]
ipq_conf = [0x90] + ipq_lmk.get_register_dump()

for i in range(len(ipq_conf)):
    if i<len(xrf_conf):
        if ipq_conf[i] != xrf_conf[i]:
            print("%x\t%x"%(ipq_conf[i], xrf_conf[i]))
    else:
        print("%x"%(ipq_conf[i]))            

In [ ]:
xrf_conf = xrfclk.xrfclk._Config['lmx2594'][491.52]
ipq_conf = [0x700000, 0x6f0000, 0x6e0000] + ipq_adc.get_register_dump()

for i in range(len(ipq_conf)):
    if i<len(xrf_conf):
        if ipq_conf[i] != xrf_conf[i]:
            print("%x\t%x"%(ipq_conf[i], xrf_conf[i]))
    else:
        print("%x"%(ipq_conf[i]))            

In [ ]:
xrfclk.xrfclk._Config['lmx2594']

In [ ]:
board = ipq_pynq_utils.ZCU208Board()
board.print_clock_summary()

In [ ]:
help(s)

In [ ]:
# s = board.spi_lmk
# addr = 388

# s.writebytes([((addr>>8) & 0xFF) + (1<<7), addr & 0xFF, 0])
# a = s.readbytes(3)
# print(a)

In [ ]:
for x in soc.clk104_gpio:
    print(x)

In [ ]:
dir(soc.clk104_gpio)

In [ ]:
dir(soc.clk104_gpio.channel1)

In [ ]:
hex(soc.clk104_gpio.channel1.trimask)

In [ ]:
help(soc.clk104_gpio)

In [ ]:
soc.clk104_gpio.mmio.array[:4]

In [ ]:
soc.clk104_gpio._registers

In [ ]:
soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x0, 0xff)

s = board.spi_adc
addr = 110
msg = [addr + (1<<7), 0, 0]
print(msg)
a = s.xfer2(msg)
print(a)

In [ ]:
while True:
    time.sleep(0.5)
    soc.clk104_gpio.channel1.setdirection('out')
    soc.clk104_gpio.channel1.write(0x1, 0xff)

    s = board.spi_dac
    addr = 110
    msg = [addr + (1<<7), 0, 0]
    print(msg)
    a = s.xfer(msg)
    print(a)

In [ ]:
s = board.spi_adc
print(s.no_cs)
print(s.cshigh)
print(s.mode)
print(s.bits_per_word)
print(s.lsbfirst)

In [ ]:
# s.cshigh = True

In [ ]:
dir(board.spi_adc)

In [ ]:
s = board.spi_lmk
addr = 388
msg = [((addr>>8) & 0xFF) + (1<<7), addr & 0xFF, 0]
print(msg)
a = s.xfer(msg)
print(a)

s = board.spi_adc
addr = 110
msg = [addr + (1<<7), 0, 0]
print(msg)
a = s.xfer3(msg)
print(a)

s = board.spi_dac
addr = 110
msg = [addr + (1<<7), 0, 0]
print(msg)
a = s.xfer3(msg)
print(a)

In [ ]:
msg

In [ ]:
clk104 = ipq_pynq_utils.clock_models.CLK104()

In [ ]:
for x in clk104.lmk.get_register_dump():
    print(hex(x))

In [ ]:
for k,v in clk104.lmk.registers_by_addr.items():
    for x in v.fields:
        if x.name != 'CONST':
            print(k, x.name)

In [ ]:
for k,v in clk104.lmx_adc.registers_by_addr.items():
    for x in v.fields:
        if x.name != 'CONST':
            print(k, x.name)

In [ ]:
help(board.spi_lmk)

In [ ]:
c = ipq_pynq_utils.clock_models.LMX2594(122.88)

In [ ]:
for x in c.get_register_dump():
    print(hex(x))

In [ ]:
c.rb_LD_VTUNE

In [ ]:
for k,v in c.registers_by_addr.items():
    for x in v.fields:
        if x.name != 'CONST':
            print(k, x.name)
#     print(k, v.fields)
    